This notebook displays on the SPAM model of sulci the value of the parapeter for each sulcus. The entry point is a csv containing the value to plot for each sulcal region.

Requires anatomist from the BrainVISA software suite

# 1. Imports 

In [11]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

In [12]:
from soma import aims
import json
import os
import pandas as pd
import numpy as np

# 2. Paths and constants

In [13]:
# Path where the models and region configuration lie
path_to_deep_folding = "/neurospin/dico/data/deep_folding/current"

# JSON file with sulcal regions containing list of sulci
# json_regions = f"{path_to_deep_folding}/sulci_regions_gridsearch.json"
# json_regions = f"{path_to_deep_folding}/sulci_regions_champollion_V1.json"
json_regions = f"{os.getcwd()}"\
    "/../../view_gene_database/"\
    "region_to_sulci_with_smaller_region_size.json"

# Path to parameter file
path_summary = f"{path_to_deep_folding}/models/Champollion_V1_after_ablation/analysis/QTIM"
path_file = "IHI_QTIM_resid_sex_age.csv"
file_to_display = f"{path_summary}/{path_file}"

# Parameters and p value
param = "auc"
p_value = "p_value"


In [14]:
# Gets SPAM models on which visualization is done
Rspam_model = aims.carto.Paths.findResourceFile(
    "models/models_2008/descriptive_models/"
    "segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg")
Lspam_model = aims.carto.Paths.findResourceFile(
    "models/models_2008/descriptive_models/segments/"
    "global_registered_spam_left/meshes/Lspam_model_meshes_1.arg")

# 2. Preprocessing

In [15]:
with open(json_regions) as f:
    regions = json.load(f)

In [16]:
# next(iter(regions[[brain]])) # Need the change to this if using the inital json file
next(iter(regions))

'F.C.L.p.-subsc.-F.C.L.a.-INSULA._left'

In [17]:
df = pd.read_csv(file_to_display)[["region", param, p_value]]
df.head()

,region,auc,p_value
0,F.Coll.-S.Rh._left,0.850495,0.000089
1,S.T.i.-S.O.T.lat._left,0.716641,0.000089
2,F.Coll.-S.Rh._right,0.696203,0.000089
3,S.T.i.-S.O.T.lat._right,0.692656,0.000089
4,F.P.O.-S.Cu.-Sc.Cal._left,0.688620,0.000089


In [18]:

res = df.groupby(['region']).mean()
res["side"] = res.index.str.split('_').str[-1]
res = res.reset_index()

In [19]:
def get_sulci(region):
    list_sulci = list(regions[f"{region}"].keys())
    return list_sulci

In [20]:
# res["region"].tolist()

In [21]:
dic = {
    "FCLp-subsc-FCLa-INSULA": "F.C.L.p.-subsc.-F.C.L.a.-INSULA.",
    "FCMpost-SpC": "F.C.M.post.-S.p.C.",
    "FColl-SRh": "F.Coll.-S.Rh.",
    "FIP": "F.I.P.",
    "FPO-SCu-ScCal": "F.P.O.-S.Cu.-Sc.Cal.",
    "Lobule_parietal_sup": "Lobule_parietal_sup.",
    "ORBITAL": "S.Or.",
    "SC-SPeC": "S.C.-S.Pe.C.",
    "SC-SPoC": "S.C.-S.Po.C.",
    "SC-sylv": "S.C.-sylv.",
    "SFinf-BROCA-SPeCinf": "S.F.inf.-BROCA-S.Pe.C.inf.",
    "SFint-FCMant": "S.F.int.-F.C.M.ant.",
    "SFint-SR": "S.F.int.-S.R.",
    "SFinter-SFsup": "S.F.inter.-S.F.sup.",
    "SFmarginal-SFinfant":"S.F.marginal-S.F.inf.ant.",
    "SFmedian-SFpoltr-SFsup": "S.F.median-S.F.pol.tr.-S.F.sup.",
    "SOr-SOlf": "S.Or.-S.Olf.",
    "SPeC": "S.Pe.C.",
    "SPoC": "S.Po.C.",
    "STi-SOTlat": "S.T.i.-S.O.T.lat.",
    "STi-STs-STpol": "S.T.i.-S.T.s.-S.T.pol.",
    "STsbr": "S.T.s.br.",
    "STs": "S.T.s.",
    "ScCal-SLi": "Sc.Cal.-S.Li.",
    "SsP-SPaint": "S.s.P.-S.Pa.int.",
}

In [22]:
for key in dic.keys():
    res["region"] = res["region"].str.replace(key, dic[key])
res.head()

,region,auc,p_value,side
0,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,0.631145,0.000179,left
1,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.566137,0.034193,right
2,F.C.M.post.-S.p.C._left,0.496190,0.558611,left
3,F.C.M.post.-S.p.C._right,0.570336,0.018302,right
4,F.Coll.-S.Rh._left,0.850495,0.000089,left


In [23]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
res = res.sort_values(by="p", ascending=False)
res = res.explode("sulcus")
res[res.region.str.contains("S.T.s.")]

In [ ]:
for _, row in res.iterrows():
    print(row.sulcus)

In [ ]:
res[res.sulcus=="S.F.orbitaire._right"]

In [ ]:
res

In [ ]:
def set_color_property(res, side):
    global dic

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    for vertex in dic[f"aims{side}"].vertices():
        vertex['p_value'] = 0.

    for _, row in res.iterrows():
        for vertex in dic[f"aims{side}"].vertices():
            vname = vertex.get('name')
            if vname == row.sulcus:
                    vertex['p_value'] = max(vertex['p_value'],
                                                row.p)
                    # if row.p < -np.log10(0.05/56):
                    #     vertex['p_value'] = 0.
                    # elif vertex['p_value'] != 0.: 
                    #     vertex['p_value'] = min(vertex['p_value'],
                    #                             row.p)
                    # else:
                    #     vertex['p_value'] = row.p
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty('p_value')
    dic[f"ana{side}"].notifyObservers()
    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])
    dic[f"ana{side}"].setPalette("green_yellow_red",
                              minVal=-np.log10(0.05/56), maxVal=10,
                              absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)


# Main function

In [ ]:
a = anatomist.Anatomist()

In [ ]:
middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]

In [ ]:
dic = {} # Global dictionary of windows

visualize_whole(res, "L", 0)
visualize_whole(res, "R", 4)